In [30]:
import folium
from folium.plugins import HeatMap
import geopandas as gpd
%store -r warsaw_tiles

In [32]:
operators_5g_coverage_warsaw = gpd.read_file('../outputs/operators_5g_coverage_warsaw.geojson')

In [47]:
orange_5g_coverage_warsaw = gpd.read_file('../outputs/orange_5g_coverage_warsaw.geojson')

In [37]:
play_5g_coverage_warsaw = operators_5g_coverage_warsaw[operators_5g_coverage_warsaw['operator'] == 'Play']
play_5g_coverage_warsaw.to_file('../outputs/play_5g_coverage_warsaw.geojson', driver='GeoJSON')

In [39]:
plus_5g_coverage_warsaw = operators_5g_coverage_warsaw[operators_5g_coverage_warsaw['operator'] == 'Plus']
plus_5g_coverage_warsaw.to_file('../outputs/plus_5g_coverage_warsaw.geojson', driver='GeoJSON')

In [41]:
tmobile_5g_coverage_warsaw = operators_5g_coverage_warsaw[operators_5g_coverage_warsaw['operator'] == 'T-Mobile']
tmobile_5g_coverage_warsaw.to_file('../outputs/tmobile_5g_coverage_warsaw.geojson', driver='GeoJSON')

In [105]:
center = [52.2297, 21.0122]
m = folium.Map(location=center, zoom_start=6)

In [107]:
folium.GeoJson(warsaw_tiles.geometry, name="Warsaw Boundary").add_to(m)

In [109]:
files = {
    'orange_5g_coverage_warsaw.geojson': 'orange',
    'play_5g_coverage_warsaw.geojson': 'purple',
    'tmobile_5g_coverage_warsaw.geojson': 'pink',
    'plus_5g_coverage_warsaw.geojson': 'green'
}

In [111]:
for file, color in files.items():
    print(f"Loading: {file}")
    gdf = gpd.read_file(f'../outputs/{file}')

    folium.GeoJson(
        gdf.geometry,
        name=file.split('_')[0].capitalize(),
        style_function=lambda x, color=color: {
            'color': color,
            'weight': 1,
            'fillOpacity': 0.2
        }
    ).add_to(m)

Loading: orange_5g_coverage_warsaw.geojson
Loading: play_5g_coverage_warsaw.geojson
Loading: tmobile_5g_coverage_warsaw.geojson
Loading: plus_5g_coverage_warsaw.geojson


In [113]:
folium.LayerControl().add_to(m)

In [115]:
output_path = '../outputs/warsaw_5g_coverage_map.html'
m.save(output_path)

print(f"Map saved successfully to {output_path}")

Map saved successfully to ../outputs/warsaw_5g_coverage_map.html


In [ ]:
##------------------------HEATMAP VISUALIZATION----------------------------------------##

In [117]:
ookla_5g_warsaw = gpd.read_file('../outputs/warsaw_coverage.geojson')

In [121]:
heat_data = [
    (row.geometry.centroid.y, row.geometry.centroid.x, row['avg_d_kbps'])
    for _, row in ookla_5g_warsaw.iterrows()
]

In [125]:
center = [52.2297, 21.0122]
m = folium.Map(location=center, zoom_start=9)

HeatMap(heat_data, radius=8, max_zoom=15).add_to(m)

output_path = '../outputs/warsaw_5g_coverage_heatmap.html'
m.save(output_path)

print(f"Map saved successfully to {output_path}")

Map saved successfully to ../outputs/warsaw_5g_coverage_heatmap.html


In [ ]:
##------------------------ORANGE VS. OTHER COMPANIES COMPARISION VISUALIZATION----------------------------------------##

In [129]:
orange_5g_coverage_warsaw = gpd.read_file('../outputs/orange_5g_coverage_warsaw.geojson')
play_5g_coverage_warsaw = gpd.read_file('../outputs/play_5g_coverage_warsaw.geojson')
plus_5g_coverage_warsaw = gpd.read_file('../outputs/plus_5g_coverage_warsaw.geojson')
tmobile_5g_coverage_warsaw = gpd.read_file('../outputs/tmobile_5g_coverage_warsaw.geojson')

In [131]:
for gdf in [orange_5g_coverage_warsaw, play_5g_coverage_warsaw, plus_5g_coverage_warsaw, tmobile_5g_coverage_warsaw]:
    gdf.geometry = gdf.geometry.simplify(tolerance=0.001, preserve_topology=True)

print("Geometries simplified for faster rendering.")

Geometries simplified for faster rendering.


In [133]:
center = [52.2297, 21.0122]
m = folium.Map(location=center, zoom_start=10)

In [135]:
def get_color(row):
    orange_speed = row['avg_d_kbps']
    competitors = {
        'Play': play_5g_coverage_warsaw,
        'Plus': plus_5g_coverage_warsaw,
        'T-Mobile': tmobile_5g_coverage_warsaw
    }
    
    max_speed = orange_speed
    best_operator = 'Orange'
    
    for name, gdf in competitors.items():
        competitor_speed = gdf.loc[gdf['cell'] == row['cell'], 'avg_d_kbps']
        if not competitor_speed.empty and competitor_speed.values[0] > max_speed:
            max_speed = competitor_speed.values[0]
            best_operator = name
    
    if best_operator == 'Orange':
        return 'green'
    elif abs(max_speed - orange_speed) <= 5000: 
        return 'yellow'
    else:
        return 'red'

In [137]:
for idx, row in orange_5g_coverage_warsaw.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda x, row=row: {
            'color': get_color(row),
            'weight': 1,
            'fillOpacity': 0.4
        },
        tooltip=f"Operator: Orange | DL Speed: {row['avg_d_kbps']} kbps"
    ).add_to(m)

In [139]:
folium.LayerControl().add_to(m)

In [141]:
m.save('../outputs/orange_others_performance_comparision_map.html')
print("Map saved to '../outputs/orange_others_performance_comparision_map.html'")

Map saved to '../outputs/orange_others_performance_comparision_map.html'
